In [5]:
from pyspark.sql.functions import *
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Tokenizer
import pyspark.sql.functions as func

In [13]:
data = spark.read.option("header", "true"). \
    option("mode", "DROPMALFORMED") \
    .csv(r"C:\Users\sgoel\PycharmProjects\University\BigData\Targil_3\Reviews_1400.csv")


In [8]:
# todo: delete
data.describe('Summary')\
    .show()

data.describe('Score')\
    .show()

data.printSchema()

data.count()


+-------+--------------------+
|summary|             Summary|
+-------+--------------------+
|  count|                1378|
|   mean|                null|
| stddev|                null|
|    min|"""Simply THE BES...|
|    max|              yummy!|
+-------+--------------------+



+-------+------------------+
|summary|             Score|
+-------+------------------+
|  count|              1378|
|   mean| 4.228592162554427|
| stddev|1.2954850003757337|
|    min|                 1|
|    max|                 5|
+-------+------------------+

root
 |-- Id: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



556089

In [9]:
data = data.select('Score', 'Summary')\
    .dropna()
data.count()

1378

In [10]:
data = data.where(length(col('Summary')) >= 75)


In [11]:
# todo: delete
data.count()


6

In [12]:
def filterData(column):

    return trim(lower(regexp_replace(column, '[^\sa-zA-Z0-9]', ''))).alias('FilteredColumn')

filteredData = data.select([filterData(data['Summary']), col("Score").alias("label")])

# נבחר לעשות  שלילי וחיובי
filteredData = filteredData.withColumn("label", when(col("label") > 80, 1.0).otherwise(0.0))

